# DATA

In [202]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [203]:
vendas = pd.read_csv('../../../data/interim/Vendas.csv', encoding='ISO-8859-1')
vendas.head()

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
0,2016.0,CAMISETA MANGA CURTA,12.0,14:54,1.0,VERBO DIVINO,7.0,00,35.5
1,2016.0,BERMUDA CICLISTA COTTON,12.0,15:08,1.0,VERBO DIVINO,1.0,08,51.0
2,2016.0,CAMISETA MANGA CURTA,12.0,15:08,1.0,VERBO DIVINO,5.0,PP,37.5
3,2016.0,BLUSA MOLETOM FLANELADA,12.0,15:27,1.0,VERBO DIVINO,1.0,M,115.0
4,2016.0,CALÇA SARJA MASC,12.0,15:27,1.0,VERBO DIVINO,1.0,GG,116.0


In [204]:
vendas = vendas.replace(['ST', 'CORSÁRIO SUPLEX', 'ESP', 'ESPECIAL'], ['00','CORSARIO SUPLEX', 'GG', 'GG'])
try:
    df_metrics.empty
except:
    df_metrics = pd.DataFrame(columns=['XGB', 'GBR'])
vendas.head()

,ANO,DESCRICAO,DIA,HORA,MES,NOME,QUANTIDADE,TAMANHO,VLR_UNIT
0,2016.0,CAMISETA MANGA CURTA,12.0,14:54,1.0,VERBO DIVINO,7.0,00,35.5
1,2016.0,BERMUDA CICLISTA COTTON,12.0,15:08,1.0,VERBO DIVINO,1.0,08,51.0
2,2016.0,CAMISETA MANGA CURTA,12.0,15:08,1.0,VERBO DIVINO,5.0,PP,37.5
3,2016.0,BLUSA MOLETOM FLANELADA,12.0,15:27,1.0,VERBO DIVINO,1.0,M,115.0
4,2016.0,CALÇA SARJA MASC,12.0,15:27,1.0,VERBO DIVINO,1.0,GG,116.0


In [205]:
df_metrics

,GBR,XGB
0,5.534733,5.576953
1,5.283639,5.318532


In [206]:
a = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'])

In [207]:
df = a.sum().reset_index().drop(['DIA', 'VLR_UNIT'], 1)
df = df.sort_values(by=['ANO','MES', 'DESCRICAO', 'TAMANHO', 'NOME'])

In [208]:
df_vlr_unit = vendas.groupby(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO']).mean()
df = df.merge(df_vlr_unit.drop(['QUANTIDADE', 'DIA'], axis=1), on=['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO'], how='left')

In [209]:
cols = df.columns.tolist()
y = cols.index('QUANTIDADE')
cols = cols[:y] + cols[y+1:] + ['QUANTIDADE']
df = df[cols]

In [210]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,M,57.5,1.0
1,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,P,57.5,1.0
2,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,VERBO DIVINO,P,57.5,1.0
3,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ALDEIA DA SERRA,PP,57.5,1.0
4,2016.0,1.0,BERMUDA CICLISTA COLEGIAL,ITAIM,PP,57.5,1.0


In [211]:
df = df.drop(df[(df.MES > 3) & (df.ANO == 2018) & (df.NOME != 'VERBO DIVINO')].index)
df = df.drop(df[(df.ANO == 2018) & (df.NOME == 'ITAIM')].index)

In [212]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

In [213]:
le_DESC = LabelEncoder()
# treino['DESCRICAO'] = le_DESC.fit_transform(treino.DESCRICAO)
# validacao['DESCRICAO'] = le_DESC.transform(validacao.DESCRICAO)
df['DESCRICAO'] = le_DESC.fit_transform(df.DESCRICAO)

In [214]:
tamanho_val = {
    '00' : 0,
    '02' : 1,
    '04' : 2,
    '06' : 3,
    '08' : 4,
    '10' : 5,
    '12' : 6,
    '14' : 7,
    'PP' : 8,
    'P' : 9,
    'M' : 10,
    'G' : 11,
    'GG' : 12
}
# treino.TAMANHO = treino.TAMANHO.apply(lambda x: tamanho_val[x])
# validacao.TAMANHO = validacao.TAMANHO.apply(lambda x: tamanho_val[x])
df.TAMANHO = df.TAMANHO.apply(lambda x: tamanho_val[x])

In [215]:
le_NOME = LabelEncoder()
# treino['NOME'] = le_NOME.fit_transform(treino.NOME)
# validacao['NOME'] = le_NOME.transform(validacao.NOME)
df['NOME'] = le_NOME.fit_transform(df.NOME)

In [216]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,0,3,10,57.5,1.0
1,2016.0,1.0,0,1,9,57.5,1.0
2,2016.0,1.0,0,3,9,57.5,1.0
3,2016.0,1.0,0,1,8,57.5,1.0
4,2016.0,1.0,0,2,8,57.5,1.0


# FEATURES

In [217]:
# descricao - mes
desc_mes = df.groupby(['DESCRICAO', 'MES']).QUANTIDADE.sum().reset_index(name='DESCRICAO_MES')
df = df.merge(desc_mes, on=['DESCRICAO', 'MES'], how='left')
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES
0,2016.0,1.0,0,3,10,57.5,1.0,16.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0


In [218]:
desc_place = df.groupby(['DESCRICAO', 'NOME']).QUANTIDADE.sum().reset_index(name='DESCRICAO_NOME')
df = df.merge(desc_place, on=['DESCRICAO', 'NOME'], how='left')
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES,DESCRICAO_NOME
0,2016.0,1.0,0,3,10,57.5,1.0,16.0,14.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0,3.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0,14.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0,3.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0,4.0


In [219]:
# year_month = df[['ANO', 'MES', 'QUANTIDADE', 'DESCRICAO', 'TAMANHO']].groupby(['ANO', 'MES', 'DESCRICAO', 'TAMANHO']).sum().reset_index()
# def get_last_year_sells(data):

#     qnt = year_month[(year_month.ANO == data['ANO'] - 1) & (year_month.MES == data['MES'])]
#     if qnt.empty:
#         return np.nan
#     else:
#         qnt = qnt[(qnt.DESCRICAO == data['DESCRICAO']) & (qnt.TAMANHO == data['TAMANHO'])].QUANTIDADE.values
#         if qnt.size > 0:
#             return qnt[0]
#         else:
#             return 0
# df['last_year_sell'] = df.apply(get_last_year_sells, axis=1)

In [220]:
# def clima(mes):
#     if mes in [1, 2, 3]:
#         return 1 #verão
#     elif mes in [4,5,6]:
#         return 2 #outono
#     elif mes in [7,8,9]:
#         return 3 #inverno
#     else:
#         return 4#primavera
# df['ESTACAO'] = df.MES.apply(clima)
# df.ESTACAO = df.ESTACAO.apply(lambda x: x > 1 and x < 4)
# est_desc = df.groupby(['ESTACAO', 'DESCRICAO']).QUANTIDADE.sum().reset_index(name='DESCRICAO_ESTACAO')
# df = df.merge(est_desc, on=['ESTACAO', 'DESCRICAO'], how='left')
# df.head()

# TRAIN

In [221]:
df.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE,DESCRICAO_MES,DESCRICAO_NOME
0,2016.0,1.0,0,3,10,57.5,1.0,16.0,14.0
1,2016.0,1.0,0,1,9,57.5,1.0,16.0,3.0
2,2016.0,1.0,0,3,9,57.5,1.0,16.0,14.0
3,2016.0,1.0,0,1,8,57.5,1.0,16.0,3.0
4,2016.0,1.0,0,2,8,57.5,1.0,16.0,4.0


In [222]:
ULTIMOS_MES = df[df.ANO == 2018].MES.max()
treino = df[(df.ANO < 2018) | (df.MES < ULTIMOS_MES)]
validacao = df[(df.ANO == 2018) & (df.MES == ULTIMOS_MES)]
Y_train = treino['QUANTIDADE']
X_train = treino.drop('QUANTIDADE', 1)
X_test = validacao.drop('QUANTIDADE', 1)
Y_test = validacao['QUANTIDADE']
# scaler = StandardScaler().fit(X_train)
# X_train_sc = scaler.transform(X_train)
# X_test_sc = scaler.transform(X_test)

In [223]:
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBRegressor
from math import sqrt

In [224]:
models = []
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBRegressor()))
# models.append(('XGB_grid', grid.best_estimator_))
results = []
new_metrics = {}
print(X_train.columns)
for name, model in models:
    if name in ['LR', 'LDA']:
        X_train_case = X_train_sc
        X_test_case = X_test_sc
    else:
        X_train_case = X_train
        X_test_case = X_test
    model.fit(X_train_case, Y_train)
    Y_pred = model.predict(X_test_case)
    metrics = sqrt(mean_squared_error(Y_test, Y_pred))
    new_metrics[name] = metrics
    print(name, metrics)
    print(model.feature_importances_)
if df_metrics.empty:
    df_metrics = df_metrics.append(new_metrics, ignore_index=True)
else:
    min_GBR = df_metrics.GBR.min()
    min_XGB = df_metrics.values.min()
#     min_grid = df_metrics.XGB_grid.min()
    if (new_metrics['GBR'] < min_GBR and new_metrics['GBR'] < min_XGB) or (new_metrics['XGB'] < min_XGB and new_metrics['XGB'] < min_GBR):
        df_new_metric = pd.DataFrame(new_metrics, index=[0])
        df_metrics = df_metrics.append(df_new_metric, ignore_index=True)

Index(['ANO', 'MES', 'DESCRICAO', 'NOME', 'TAMANHO', 'VLR_UNIT',
       'DESCRICAO_MES', 'DESCRICAO_NOME'],
      dtype='object')
GBR 5.581969596761228
[0.02059962 0.06305475 0.02077654 0.15970192 0.23806653 0.07708022
 0.26775314 0.1529673 ]
XGB 5.499873743987366
[0.03       0.09142857 0.03428571 0.19       0.22142857 0.08
 0.23428571 0.11857143]


In [225]:
param = {'max_depth': 7, 
         'subsample': .85, 
         'colsample_bylevel': 1,
         'n_estimators': 200, 
         'learning_rate': .025,
         'random_state': 40,  # 1
         'silent':0,
         'reg_alpha':.1,
         'eval_metric':'rmse',
        }

clf_xgb = XGBRegressor(**param)
clf_xgb.fit(X_train.values, Y_train.values, eval_set=[(X_test.values, Y_test.values)], early_stopping_rounds=10)

[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 168 extra nodes, 0 pruned nodes, max_depth=7
[0]	validation_0-rmse:9.91646
Will train until validation_0-rmse hasn't improved in 10 rounds.
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=7
[1]	validation_0-rmse:9.79146
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 184 extra nodes, 0 pruned nodes, max_depth=7
[2]	validation_0-rmse:9.67528
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra nodes, 0 pruned nodes, max_depth=7
[3]	validation_0-rmse:9.56039
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 174 extra nodes, 0 pruned nodes, max_depth=7
[4]	validation_0-rmse:9.44455
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 170 extra nodes, 0 pruned nodes, max_depth=7
[5]	validation_0-rmse:9.3896
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 188 extra 

[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[57]	validation_0-rmse:6.80399
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=7
[58]	validation_0-rmse:6.77727
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 206 extra nodes, 0 pruned nodes, max_depth=7
[59]	validation_0-rmse:6.76192
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=7
[60]	validation_0-rmse:6.7561
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 0 pruned nodes, max_depth=7
[61]	validation_0-rmse:6.74382
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 224 extra nodes, 0 pruned nodes, max_depth=7
[62]	validation_0-rmse:6.72369
[00:44:50] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 232 extra nodes, 0 pruned nodes, max_depth=7
[63]	validation_0-rmse:6

[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 204 extra nodes, 0 pruned nodes, max_depth=7
[114]	validation_0-rmse:6.41807
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[115]	validation_0-rmse:6.41462
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 208 extra nodes, 0 pruned nodes, max_depth=7
[116]	validation_0-rmse:6.41945
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 222 extra nodes, 0 pruned nodes, max_depth=7
[117]	validation_0-rmse:6.41332
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 198 extra nodes, 0 pruned nodes, max_depth=7
[118]	validation_0-rmse:6.3963
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 216 extra nodes, 0 pruned nodes, max_depth=7
[119]	validation_0-rmse:6.39057
[00:44:51] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 226 extra nodes, 0 pruned nodes, max_depth=7
[120]	validation_0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='rmse', gamma=0,
       learning_rate=0.025, max_delta_step=0, max_depth=7,
       min_child_weight=1, missing=None, n_estimators=200, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=40,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=0, subsample=0.85)

In [226]:
Y_pred = clf_xgb.predict(X_test.values)
sqrt(mean_squared_error(Y_test, Y_pred))

6.378996000007701

In [227]:
df_metrics

,GBR,XGB
0,5.534733,5.576953
1,5.283639,5.318532


In [228]:
models = []
models.append(('GBR', GradientBoostingRegressor()))
models.append(('XGB', XGBRegressor()))
# models.append(('LR', LogisticRegression()))
# models.append(('LDA', LinearDiscriminantAnalysis()))
# ?models.append(('XGB_grid', grid.best_estimator_))
for name, model in models:
    metrics = []
    for year in df.ANO.unique()[1:]:
        treino = df[df.ANO == year -1].drop('ANO', axis=1)
        validacao = df[df.ANO == year].drop('ANO', axis=1)
        Y_train = treino['QUANTIDADE']
        X_train = treino.drop('QUANTIDADE', 1)
        X_test = validacao.drop('QUANTIDADE', 1)
        Y_test = validacao['QUANTIDADE']
        if name in ['LR', 'LDA']:
            scaler = StandardScaler().fit(X_train)
            X_train = scaler.transform(X_train)
            X_test = scaler.transform(X_test)
        model.fit(X_train, Y_train)
        Y_pred = model.predict(X_test)
        metrics.append(sqrt(mean_squared_error(Y_test, Y_pred)))
    print(name, np.mean(metrics))

GBR 6.263029204811843
XGB 6.285253345473589


# XGBOOST GRID SEARCH

In [229]:
XGBRegressor()

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [230]:
params = {
        'max_depth' : [3,7],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'n_estimators' : [500, 600, 400]
        }

In [231]:
XGB = XGBRegressor(learning_rate=.01, nthread=1)

In [232]:
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold, GridSearchCV

In [233]:
folds = 5
param_comb = 20
skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)
random_search = RandomizedSearchCV(
    XGB, param_distributions=params, n_iter=param_comb, scoring='neg_mean_squared_error',
    n_jobs=4, cv=skf.split(X_train,Y_train), verbose=3, random_state=1001 )

In [97]:
random_search.fit(X_train, Y_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/chazanas/.virtualenvs/sold_alt/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-32.904173404263695, total=   0.7s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-300.5451710071674, total=   0.7s
[CV] subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0 
[CV] subsample=0.6, min_child_weight=1, max_depth=5, gamma=1.5, colsample_bytree=0.8 
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-25.201480612471226, total=   0.6s
[CV]  subsample=1.0, min_child_weight=5, max_depth=3, gamma=5, colsample_bytree=1.0, score=-26.259217432254218, total

[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    7.5s


[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-27.378083579824967, total=   0.7s
[CV] subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=0.8, min_child_weight=1, max_depth=4, gamma=1, colsample_bytree=1.0, score=-32.66564149850497, total=   1.1s
[CV] subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-271.051621442313, total=   0.7s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-26.14852362738454, total=   0.8s
[CV] subsample=1.0, min_child_weight=1, max_depth=5, gamma=5, colsample_bytree=0.6 
[CV]  subsample=1.0, min_child_weight=10, max_depth=4, gamma=1.5, colsample_bytree=0.6, score=-23.21251614305254, total=   0.7s
[CV] subsample=1.0, m

[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=-21.031789511953733, total=   1.3s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=0.6, min_child_weight=10, max_depth=5, gamma=1, colsample_bytree=0.6, score=-27.78262004111416, total=   1.2s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-24.21904912922863, total=   1.4s
[CV] subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0 
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-187.72210253650698, total=   1.4s
[CV]  subsample=1.0, min_child_weight=5, max_depth=5, gamma=0.5, colsample_bytree=1.0, score=-21.82489947365507, total=   1.3s
[CV] subsample=1.0, m

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   30.9s finished


RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x10ff13200>,
          error_score='raise',
          estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.02, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=600,
       n_jobs=1, nthread=1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          fit_params=None, iid=True, n_iter=20, n_jobs=4,
          param_distributions={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=1001, refit=True,
          return_train_score='warn', scoring='neg_mean_squared_error',
          verbose=3)

In [234]:
grid = GridSearchCV(estimator=XGB, param_grid=params, scoring='neg_mean_squared_error', n_jobs=4, cv=skf.split(X_train,Y_train), verbose=4 )
grid.fit(X_train, Y_train)

Fitting 5 folds for each of 1215 candidates, totalling 6075 fits
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6, score=-17.01512527068247, total=   0.4s
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6, score=-15.579628588894881, total=   0.4s
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=500, subsample=0.6, score=-331.18245329781536, total=   0.4s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimator

[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    2.2s


[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.6, score=-60.69053638665228, total=   0.5s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.8, score=-17.40599822948234, total=   0.5s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.8 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.8, score=-318.47569969526637, total=   0.5s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=1.0 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=0.8, score=-14.716121008504889, total=   0.4s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=1, n_estimators=600, subsample=1.0 
[C

[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=0.8, score=-16.466460580554493, total=   0.9s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=1.0 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=1.0, score=-17.13206139282984, total=   0.7s
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=0.8, score=-47.189112297938934, total=   0.8s
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=0.8, score=-52.94828740151235, total=   0.8s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=1.0 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=1.0 
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=500, subsample=1.0 
[C

[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=400, subsample=1.0, score=-363.91256003266943, total=   0.3s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=400, subsample=1.0 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=400, subsample=0.8, score=-48.744219850472476, total=   0.4s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, n_estimators=500, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=400, subsample=1.0, score=-16.527836552763606, total=   0.3s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, n_estimators=500, subsample=0.6 
[CV]  colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=5, n_estimators=400, subsample=1.0, score=-43.54317659715484, total=   0.3s
[CV] colsample_bytree=0.6, gamma=0.5, max_depth=3, min_child_weight=10, n_estimators=500, subsample=0.6

...

[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.6, score=-57.83189268155039, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.8, score=-17.267664160875903, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.8 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.8, score=-281.3211598881723, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=0.8, score=-10.646838708011265, total=   0.4s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=1, n_estimators=400, subsample=1.0 
[CV]  colsample_by

[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=0.8, score=-42.38087319979805, total=   0.7s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0, score=-21.49305286528209, total=   0.6s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=0.8, score=-71.89649452894642, total=   0.7s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=5, n_estimators=600, subsample=1.0, score=-303.17871243424645, total=   0.6s
[CV] colsample_bytr

[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=500, subsample=1.0, score=-328.9549692245457, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=600, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=500, subsample=1.0, score=-13.602005306363406, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=600, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=500, subsample=1.0, score=-50.875517897616945, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=600, subsample=0.6 
[CV]  colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=500, subsample=1.0, score=-60.02402481728115, total=   0.5s
[CV] colsample_bytree=1.0, gamma=5, max_depth=5, min_child_weight=10, n_estimators=600, subsample=0.6 
[CV]  cols

[Parallel(n_jobs=4)]: Done 6075 out of 6075 | elapsed: 13.4min finished


GridSearchCV(cv=<generator object _BaseKFold.split at 0x116b9faf0>,
       error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'max_depth': [3, 4, 5], 'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'n_estimators': [500, 600, 400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=4)

In [235]:
XGB = grid.best_estimator_
XGB.fit(X_train, Y_train)
Y_pred = XGB.predict(X_test)
sqrt(mean_squared_error(Y_test, Y_pred))

6.16427132752525

In [236]:
from sklearn.externals import joblib

In [237]:
joblib.dump(XGB, 'gridXGB.dat')

['gridXGB.dat']

# TESTES

In [28]:
vendas.QUANTIDADE.unique()

array([ 7.,  1.,  5.,  6.,  2.,  3.,  4., 12.,  8.,  9., 10.])

In [85]:
treino.head()

,ANO,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,QUANTIDADE
0,2016.0,1.0,0,3,10,57.5,1.0
1,2016.0,1.0,0,1,9,57.5,1.0
2,2016.0,1.0,0,3,9,57.5,1.0
3,2016.0,1.0,0,1,8,57.5,1.0
4,2016.0,1.0,0,2,8,57.5,1.0


In [171]:
df_metrics.empty

True

In [173]:
df.ANO.min()

2016.0

In [49]:
grid

GridSearchCV(cv=<generator object _BaseKFold.split at 0x11219ef10>,
       error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.01, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'min_child_weight': [1, 5, 10], 'gamma': [0.5, 1, 1.5, 2, 5], 'subsample': [0.6, 0.8, 1.0], 'colsample_bytree': [0.6, 0.8, 1.0], 'max_depth': [3, 4, 5], 'n_estimators': [500, 600, 400]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=4)

In [13]:
s = pd.Series({'a' : 1, 'b' : 4})
s.shift(-1)

a    4.0
b    NaN
dtype: float64

In [62]:
X_train.head()

,MES,DESCRICAO,NOME,TAMANHO,VLR_UNIT,DESCRICAO_MES,DESCRICAO_NOME,last_year_sell
0,1.0,0,3,10,57.5,16.0,14.0,NaN
1,1.0,0,1,9,57.5,16.0,3.0,NaN
2,1.0,0,3,9,57.5,16.0,14.0,NaN
3,1.0,0,1,8,57.5,16.0,3.0,NaN
4,1.0,0,2,8,57.5,16.0,4.0,NaN
